In [1]:
# Import libraries
import mysql.connector

In [2]:
def printList(arr):
    for i in arr:
        print(i)

In [3]:
# Establish connection to MySQL and create Database

try:
    db = mysql.connector.connect(host ="localhost",
                                user = "root",
                               password="juancs123")
    c = db.cursor()

    c.execute("CREATE DATABASE seleksi_basdat")

    db.close()
    
except:
    print("Error while connecting to MySQL")


In [4]:
# Use specific database
db = mysql.connector.connect(host="localhost", user="root", password="juancs123", database="seleksi_basdat")

c = db.cursor()
c.execute("ALTER DATABASE seleksi_basdat CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci")

In [5]:
# Create tables
product_create = """ CREATE TABLE `seleksi_basdat`.`product` (
    `id` INT NOT NULL AUTO_INCREMENT,
    `listing_id` BIGINT NOT NULL,
    `title` VARCHAR(255) NOT NULL,
    `type` VARCHAR(50) NOT NULL,
    `available` VARCHAR(50),
    `sold` INT,
    `return` VARCHAR(50),
    `seller` VARCHAR(100) NOT NULL,
    `condition` VARCHAR(50) NOT NULL,
    PRIMARY KEY (`id`)
)"""

price_create = """ CREATE TABLE `seleksi_basdat`.`price` (
    `id` INT REFERENCES `product`(`id`),
    `price_original_currency` VARCHAR(10) NOT NULL,
    `price_original_value` FLOAT NOT NULL,
    `price_idr` FLOAT NOT NULL,
    PRIMARY KEY (`id`)
)"""

shipment_create = """ CREATE TABLE `seleksi_basdat`.`shipment` (
    `id` INT REFERENCES `product`(`id`),
    `shipping_availability` BIT NOT NULL,
    `shipping_price_currency` VARCHAR(10),
    `shipping_price_value` FLOAT,
    `shipping_location` VARCHAR(100),
    PRIMARY KEY (`id`)
)"""

c = db.cursor()
c.execute(product_create)
c = db.cursor()
c.execute(price_create)
c = db.cursor()
c.execute(shipment_create)


In [6]:
# import preprocessed data as dataframe
import pandas as pd
import numpy as np
import os

# import the json file
dir = os.getcwd()
print(dir[:-3]+"data")
df = pd.read_json(dir[:-3]+"data\data_preprocessed.json", orient="index")
df.head()



d:\Folder Kuliah Cadangan\UKM and Others\Aslab\Basdat\Seleksi-2023-Tugas-1\Data Scraping\data


,listing_id,title,type,available,sold,price_idr,shipping_location,return,seller,condition,price_original_currency,price_original_value,shipping_availability,shipping_price_currency,shipping_price_value
0,354907151520,"Microsoft 13"" Surface Pro 8 (i5, 8GB RAM, 256G...",Laptops & Netbooks,Limited quantity,1.0,9999850.75,"Avenel, New Jersey, United States",30 days,Always Deals Store,New,US $,669.99,True,US $,54.46
1,333563280818,CHUWI Laptop HeroBook GemiBook CoreBook Pro/Pl...,Laptops & Netbooks,Limited quantity,32.0,4026996.38,None,30 days,CHUWI Official Store（chuwi2018）,New,EUR,239.99,True,None,0.00
2,333996205343,"CHUWI HeroBook Pro 14.1"" Windows 10 Laptop Int...",Laptops & Netbooks,Limited quantity,14.0,3355802.35,None,30 days,CHUWI Official Store（chuwi2018）,New,EUR,199.99,True,None,0.00
3,295671743764,"HP Omen 17.3"" Gaming Notebook QHD 165Hz i9-139...",Laptops & Netbooks,Limited quantity,92.0,37313283.58,"Atlanta, Georgia, United States",30 days,Antonline,New,US $,2499.99,False,None,NaN
4,285185433173,"Lenovo ThinkPad P1 Gen 5 15.6"" 2022 2K 2.3GHz ...",Laptops & Netbooks,Limited quantity,5.0,20895373.13,"Sanford, Florida, United States",30 days,ItsWorthMore,Excellent - Refurbished,US $,1399.99,True,US $,168.15


In [8]:
# Insert data to product table
c = db.cursor()

product_insert = """ INSERT INTO `seleksi_basdat`.`product` 
    (`listing_id`, `title`, `type`, `available`, `sold`, `return`, `seller`, `condition`)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

data = list()

for i in range(len(df)):
    sold_attrib = df.iloc[i,4]
    if ((sold_attrib is not None) and (not pd.isna(sold_attrib))):
        sold_attrib = int(sold_attrib)
    elif (pd.isna(sold_attrib)):
        sold_attrib = None
    tempTuple = (int(df.iloc[i,0]), df.iloc[i,1], df.iloc[i,2], df.iloc[i,3], sold_attrib, df.iloc[i,7], df.iloc[i,8], df.iloc[i,9])
    data.append(tempTuple)


c.executemany(product_insert, data)
db.commit()

In [ ]:
# Insert data to price table

price_insert = """ INSERT INTO `seleksi_basdat`.`price` 
    (`id`, `price_original_currency`, `price_original_value`, `price_idr`)
    VALUES (%s, %s, %s, %s)
"""

get_product_id = """ SELECT `id` FROM product 
    WHERE product.listing_id = %s """

data = list()

for i in range(len(df)):
    c = db.cursor()
    c.execute(get_product_id, (int(df.iloc[i,0]), ))
    id = c.fetchall()[0][0]

    tempTuple = (id, df.iloc[i,10], df.iloc[i,11], df.iloc[i,5])
    data.append(tempTuple)

c = db.cursor()
c.executemany(price_insert, data)
db.commit()

In [ ]:
# Insert data to shipment table
shipment_insert = """ INSERT INTO `seleksi_basdat`.`shipment` 
    (`id`, `shipping_availability`, `shipping_price_currency`, `shipping_price_value`, `shipping_location`)
    VALUES (%s, %s, %s, %s, %s)
"""

get_product_id = """ SELECT `id` FROM product 
    WHERE product.listing_id = %s """

data = list()

for i in range(len(df)):
    c = db.cursor()
    c.execute(get_product_id, (int(df.iloc[i,0]), ))
    id = c.fetchall()[0][0]

    if (df.iloc[i,12]):
        availability = 1
    else :
        availability = 0

    ship_price = df.iloc[i,14]
    if (pd.isna(ship_price)):
        ship_price = None

    tempTuple = (id, availability, df.iloc[i,13], ship_price, df.iloc[i,6])
    data.append(tempTuple)


c = db.cursor()
c.executemany(shipment_insert, data)
db.commit()